In [1]:
import pandas as pd

### Get a fraction of the original dataset 

In [2]:
original_data = pd.read_csv("../abc_data.csv")

original_data['publish_date'] = pd.to_datetime(original_data['publish_date'], format='%Y%m%d')
reduced_data = original_data[original_data['publish_date'].dt.year > 2018]

reduced_data = reduced_data.groupby(pd.Grouper(key='publish_date', freq='M')).apply(lambda x: x.sample(frac=0.1))
reduced_data = reduced_data.reset_index(drop=True)

reduced_data.to_csv("../abc_reduced_data.csv", index=False)
reduced_data.head()

publish_date                                      headline_text
0   2019-01-20                      thousands attend womens march
1   2019-01-22  former socceroo craig foster visits refugee ha...
2   2019-01-14  media attacked outside court as kangaroo tortu...
3   2019-01-28  ambulance officer dies attending emergency in ...
4   2019-01-25  florida official resigns over hurricane katrin...

### Check if percentage ratio per month are the same after row deletion 

In [3]:
def process_csv(input_file, date_format):
    data = pd.read_csv(input_file)

    data['publish_date'] = pd.to_datetime(data['publish_date'], format=date_format)
    data = data[data['publish_date'].dt.year > 2018]

    data['date'] = data['publish_date'].dt.to_period('M')

    month_counts = data.groupby('date')['headline_text'].count().reset_index()

    total_headlines = month_counts['headline_text'].sum()

    month_counts['percentage'] = (month_counts['headline_text'] / total_headlines * 100).round(2)
    month_counts.columns = ['date', 'headline_count', 'percentage']

    return month_counts

original_data = process_csv('../abc_data.csv', '%Y%m%d')
reduced_data = process_csv('../abc_reduced_data.csv', '%Y-%m-%d')

merged_data = pd.concat([reduced_data['date'], original_data['headline_count'], reduced_data['headline_count'], 
                        original_data['percentage'], reduced_data['percentage']], axis=1)

merged_data.columns = ['date', 'original_count', 'reduced_count', 'original_percentage', 'reduced_percentage']
merged_data.head()


date  original_count  reduced_count  original_percentage  \
0  2019-01            2607            261                 2.83   
1  2019-02            2838            284                 3.08   
2  2019-03            3005            300                 3.26   
3  2019-04            2773            277                 3.01   
4  2019-05            3165            316                 3.43   

   reduced_percentage  
0                2.83  
1                3.08  
2                3.25  
3                3.00  
4                3.43